# Getting Started

In [1]:
%reload_ext autoreload
%autoreload 2
from recsys.MovieLens import MovieLens
from surprise import SVD

In [2]:
def create_anti_test_set(user_id, trainset):
    fill = trainset.global_mean
    anti_testset = []
    inner_uid = trainset.to_inner_uid(str(user_id))
    
    user_items = set([j for (j, _) in trainset.ur[inner_uid]])
    anti_testset += [(trainset.to_raw_uid(inner_uid), trainset.to_raw_iid(i), fill) for
                             i in trainset.all_items() if
                             i not in user_items]
    return anti_testset

In [ ]:
# Pick an arbitrary test subject
test_subject = 85

lens, data, _ = MovieLens.load()

user_ratings = lens.get_user_ratings(test_subject)
loved = []
hated = []
for ratings in user_ratings:
    if (float(ratings[1]) > 4.0):
        loved.append(ratings)
    if (float(ratings[1]) < 3.0):
        hated.append(ratings)

print("\nUser ", test_subject, " loved these movies:")
for ratings in loved:
    print(lens.get_movie_name(ratings[0]))
print("\n...and didn't like these movies:")
for ratings in hated:
    print(lens.get_movie_name(ratings[0]))

In [ ]:
# Building recommendation model
train_set = data.build_full_trainset()

algo = SVD()
algo.fit(train_set)

In [ ]:
# Computing recommendations
test_set = create_anti_test_set(test_subject, train_set)
predictions = algo.test(test_set)

recommendations = []

print("Loading movie ratings...")
data = lens.load_movielens_data()

print ("\nWe recommend:")
for user_id, movie_id, true_rating, predicted_rating, _ in predictions:
    intMovieID = int(movie_id)
    recommendations.append((intMovieID, predicted_rating))

recommendations.sort(key=lambda x: x[1], reverse=True)

for ratings in recommendations[:10]:
    print(lens.get_movie_name(ratings[0]))